# 1、概要

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread, imshow
import cv2

%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from subprocess import check_output
print(check_output(["ls", "../input/train"]).decode("utf8"))

ModuleNotFoundError: No module named 'skimage'

# 2、准备数据

查看目标变量的分布。当分布不平衡时，根据评分标准和具体模型的使用不同，可能会严重影响性能。

对 Numerical Variable，可以用 Box Plot 来直观地查看它的分布。

对于坐标类数据，可以用 Scatter Plot 来查看它们的分布趋势和是否有离群点的存在。

对于分类问题，将数据根据 Label 的不同着不同的颜色绘制出来，这对 Feature 的构造很有帮助。

绘制变量之间两两的分布和相关度图表。

In [ ]:
from glob import glob
basepath = '../input/train/'

all_cervix_images = []

for path in sorted(glob(basepath + "*")):
    cervix_type = path.split("/")[-1]
    cervix_images = sorted(glob(basepath + cervix_type + "/*"))
    all_cervix_images = all_cervix_images + cervix_images

all_cervix_images = pd.DataFrame({'imagepath': all_cervix_images})
all_cervix_images['filetype'] = all_cervix_images.apply(lambda row: row.imagepath.split(".")[-1], axis=1)
all_cervix_images['type'] = all_cervix_images.apply(lambda row: row.imagepath.split("/")[-2], axis=1)
all_cervix_images.head()

## 2.1 图像类型

现在我们有了一个方便的dataframe数据，我们可以对数据进行一些聚合。让我们先看看每个子宫颈类型和它们的文件类型有多少个图像。

所有的文件都是JPG格式的，2型是最常见的一种，在训练数据中，总共有略多于50%的数据，1型在训练数据中略少于20%。

In [ ]:
print('We have a total of {} images in the whole dataset'.format(all_cervix_images.shape[0]))
type_aggregation = all_cervix_images.groupby(['type', 'filetype']).agg('count')  # agg：聚合
type_aggregation_p = type_aggregation.apply(lambda row: 1.0*row['imagepath']/all_cervix_images.shape[0], axis=1)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))

type_aggregation.plot.barh(ax=axes[0])  # barh:水平条形图
axes[0].set_xlabel("image count")
type_aggregation_p.plot.barh(ax=axes[1])
axes[1].set_xlabel("training size fraction")

查看每个类型的文件，以了解图像的外观。

这些图像在样式上似乎有所不同，前两个样本只有一个圆形区域和实际图像，最后一个样本在一个矩形中有图像。

In [ ]:
fig = plt.figure(figsize=(12,8))

i = 1
for t in all_cervix_images['type'].unique():
    ax = fig.add_subplot(1,3,i)
    i += 1
    f = all_cervix_images[all_cervix_images['type'] == t]['imagepath'].values[0]
    plt.imshow(plt.imread(f))
    plt.title('sample for cervix {}'.format(t))

## 2.2 图像尺寸

In [ ]:
查看下有多少种不同尺寸（shape）的图像。为了减少运行时间，每个类只需要一个子样本。

In [ ]:
from collections import defaultdict  # 使用dict时，如果引用的Key不存在，就会抛出KeyError。如果希望key不存在时，返回一个默认值，就可以用defaultdict

images = defaultdict(list)

for t in all_cervix_images['type'].unique():
    sample_counter = 0
    for _, row in all_cervix_images[all_cervix_images['type'] == t].iterrows():
        #print('reading image {}'.format(row.imagepath))
        try:
            img = imread(row.imagepath)
            sample_counter +=1
            images[t].append(img)
        except:
            print('image read failed for {}'.format(row.imagepath))
        if sample_counter > 35:  # 每种图取36幅，总共取108幅
            break

In [ ]:
dfs = []
for t in all_cervix_images['type'].unique():
    t_ = pd.DataFrame(
        {
            'nrows': list(map(lambda i: i.shape[0], images[t])),
            'ncols': list(map(lambda i: i.shape[1], images[t])),
            'nchans': list(map(lambda i: i.shape[2], images[t])),
            'type': t
        }
    )
    dfs.append(t_)

shapes_df = pd.concat(dfs, axis=0)  # 合并
# 获取每种图各尺寸的数目统计
shapes_df_grouped = shapes_df.groupby(by=['nchans', 'ncols', 'nrows', 'type']).size().reset_index().sort_values(['type', 0], ascending=False)
shapes_df_grouped

样本中的所有图像都有三个通道，现在可以忽略这些信息。建立一个barplot，通过子宫颈类型了解图像尺寸的分布。

In [ ]:
shapes_df_grouped['size_with_type'] = shapes_df_grouped.apply(lambda row: '{}-{}-{}'.format(row.ncols, row.nrows, row.type), axis=1)
shapes_df_grouped = shapes_df_grouped.set_index(shapes_df_grouped['size_with_type'].values)
shapes_df_grouped['count'] = shapes_df_grouped[[0]]

plt.figure(figsize=(10,8))
#shapes_df_grouped['count'].plot.barh(figsize=(10,8))  # pandas(为什么不需要指定y？)
sns.barplot(x="count", y="size_with_type", data=shapes_df_grouped)  # seaborn

# 3、数据分析

## 3.1 TSNE embedding（数据降维）

**TSNE(t分布随机相邻嵌入)方法相比PCA等线性降维方法，能有效将数据投影到低维空间并保持严格的分割界面（维持距离）;缺点是计算复杂度大,一般推荐先线性降维然后再用t-SNE降维。**

现在我们将获取所有的示例图像，重新缩放它们并将它们转换为灰度图像。结果将产生一个矩阵，其中每一行都是灰度图像的扁平像素。

由于原始图像是高分辨率的，因此将尺寸范围缩小到100*100将导致信息的大量丢失。因此降维到两个维度可能不会具有良好的结构，这可以通过单独观察宫颈癌类型的分布看到。

同时，我们在每个阶段中只提供很少的图像，TSNE可以用来在这些图像上做一个维持距离的降维。

我添加了一个选项，选择将它们转换为灰度，然后将它们传递给TSNE。每一个默认图像将保留其RGB信息，因此将被转换为100*100*3 = 30000维向量。

In [ ]:
def transform_image(img, rescaled_dim, to_gray=False):
    resized = cv2.resize(img, (rescaled_dim, rescaled_dim), cv2.INTER_LINEAR) # INTER_LINEAR：线性插值

    if to_gray:
        resized = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY).astype('float')
    else:
        resized = resized.astype('float')

    normalized = cv2.normalize(resized, None, 0.0, 1.0, cv2.NORM_MINMAX) # NORM_MINMAX：线性归一化
    timg = normalized.reshape(1, np.prod(normalized.shape)) # 连乘

    return timg / np.linalg.norm(timg)  # L2

rescaled_dim = 100

all_images = []
all_image_types = []

for t in all_cervix_images['type'].unique():
    all_images = all_images + images[t]
    all_image_types = all_image_types + len(images[t])*[t]

# - normalize each uint8 image to the value interval [0, 1] as float image
# - rgb to gray
# - downsample image to rescaled_dim X rescaled_dim
# - L2 norm of each sample = 1
gray_all_images_as_vecs = [transform_image(img, rescaled_dim) for img in all_images]  # list
print(np.array(gray_all_images_as_vecs).shape)  # (108, 1, 30000)

gray_imgs_mat = np.array(gray_all_images_as_vecs).squeeze()  # 压缩维度（移除长度为1的轴），能将矩阵变向量
all_image_types = np.array(all_image_types)
gray_imgs_mat.shape, all_image_types.shape

**3D t-SNE宫颈指标**
现在让我们将100*100*3图像投影到三维图像上，以检查低维图案。

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(
    n_components=3, # 嵌入空间的维数
    init='random', # pca
    random_state=101,
    method='barnes_hut',
    n_iter=500,
    verbose=2
).fit_transform(gray_imgs_mat)

In [ ]:
from sklearn import preprocessing

# https://plot.ly/python/3d-scatter-plots/
trace1 = go.Scatter3d(
    x=tsne[:,0],
    y=tsne[:,1],
    z=tsne[:,2],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        color = preprocessing.LabelEncoder().fit_transform(all_image_types),
        colorscale = 'Portland',
        colorbar = dict(title = 'cervix types'),
        line=dict(color='rgb(255, 255, 255)'),
        opacity=0.9
    )
)

data=[trace1]
layout=dict(height=800, width=800, title='3D embedding of images')
fig=dict(data=data, layout=layout)
py.iplot(fig, filename='3DBubble')

可以清楚地看到，有一个大的各种各样的聚类图和一些非常遥远的离群值。

In [ ]:
for t in all_cervix_images['type'].unique():
    tsne_t = tsne[np.where(all_image_types == t), :][0]
    plt.scatter(tsne_t[:, 0], tsne_t[:, 1]) # 散点图
plt.legend(all_cervix_images['type'].unique())  # 显示图例

**TSNE 用作图像聚类**

In [ ]:
现在让我们使用压缩后的图像和TSNE的保持距离转换来理解图集。

为此，我们将使用matplotlib来替换先前的红、绿和蓝色的点，并将实际图像输入到TSNE转换中。

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
def imscatter(x, y, images, ax=None, zoom=0.01):
    ax = plt.gca()
    images = [OffsetImage(image, zoom=zoom) for image in images] # 缩放图
    artists = []
    for x0, y0, im0 in zip(x, y, images):
        ab = AnnotationBbox(im0, (x0, y0), xycoords='data', frameon=False) # 标注框
        artists.append(ax.add_artist(ab))
    ax.update_datalim(np.column_stack([x, y])) # 堆叠显示
    ax.autoscale()
    #return artists

nimgs = 60
plt.figure(figsize=(10,8))
imscatter(tsne[0:nimgs,0], tsne[0:nimgs,1], all_images[0:nimgs]) # 坐标里绘制图像

## 3.2 成对图像距离的聚类

从纯数值的角度来看图像是如何相互联系的，现在我们来看看成对图像的距离。为此，我们将使用scipy的pdist方法。

黄色的聚集区域告诉我们有一些图像与我们读取的训练图像样本中的所有其他图像具有相对较高的距离。在clustermap的左侧和顶部，我们为每行每列找到了三种颜色之一，该颜色表示子宫颈的类型。

In [ ]:
pal = sns.color_palette("hls", 3)  # 调色板。HLS：Hue(色相)、Luminance(亮度)、Saturation(饱和度)
sns.palplot(pal)

In [ ]:
from scipy.spatial.distance import pdist, squareform

sq_dists = squareform(pdist(gray_imgs_mat))  # pdist：计算成对图像的矩阵距离，squareform：转换为对称矩阵，对角线为0（自己和自己的距离）

all_image_types = list(all_image_types)

d = {
    'Type_1': pal[0], # Red
    'Type_2': pal[1], # Green
    'Type_3': pal[2] # Blue
}

# translate each sample to its color
colors = list(map(lambda t: d[t], all_image_types))

# http://seaborn.pydata.org/generated/seaborn.clustermap.html?highlight=clustermap#seaborn.clustermap
sns.clustermap(  # 分层热图
    sq_dists,
    figsize=(12,12),
    row_colors=colors, col_colors=colors,
    cmap=plt.get_cmap('viridis')
)

In [ ]:
# 这是非聚合的距离矩阵

mask = np.zeros_like(sq_dists, dtype=np.bool)  # 返回和输入矩阵类似但用0填充的矩阵
mask[np.triu_indices_from(mask)] = True  # 返回上三角矩阵的索引

plt.figure(figsize=(12,12))
sns.heatmap(sq_dists, cmap=plt.get_cmap('viridis'), square=True, mask=mask)

**图像近邻**

给定宫颈图像库和上面的相似性矩阵，我们现在将绘制相似性矩阵的极端情况。

为此，我们找到具有以下内容的图像：
- 与所有其他图像相比，具有平均最大距离的图像
- 与所有其他图像相比，具有平均最小距离的图像
- 并将这两个图像与来自一组的平均图像进行比较。

In [ ]:
# upper triangle of matrix set to np.nan
sq_dists[np.triu_indices_from(mask)] = np.nan
sq_dists[0, 0] = np.nan

fig = plt.figure(figsize=(12,8))
# maximally dissimilar image
ax = fig.add_subplot(1,3,1)
maximally_dissimilar_image_idx = np.nanargmax(np.nanmean(sq_dists, axis=1))
plt.imshow(all_images[maximally_dissimilar_image_idx])
plt.title('maximally dissimilar')

# maximally similar image
ax = fig.add_subplot(1,3,2)
maximally_similar_image_idx = np.nanargmin(np.nanmean(sq_dists, axis=1))
plt.imshow(all_images[maximally_similar_image_idx])
plt.title('maximally similar')

# now compute the mean image
ax = fig.add_subplot(1,3,3)
mean_img = gray_imgs_mat.mean(axis=0).reshape(rescaled_dim, rescaled_dim, 3)
plt.imshow(cv2.normalize(mean_img, None, 0.0, 1.0, cv2.NORM_MINMAX))
plt.title('mean image')

In [ ]:
左边的图像有很多蓝色，我不希望它出现在大多数样本中。我认为这是“最大不同图像”的一个很好的描述方式。

# 4、模型选择

现在我们已经建立了关于数据的基本概念，接下来用最简单的方法，我们将调整彩色图像的大小和标签，并训练一个简单的线性模型，如逻辑回归。

理解这一点非常重要，我们只查看了一些训练实例，108个，并有数千个维度。为了能够应对，我们很可能会使用L1正则化。

对于我们所面临的多类问题，我们将使用**OVR的标准方法(one vs rest)**，这意味着我们将训练三个模型，其中每个模型分别用于区分第1类、第2类和第3类。

## 4.1 模型训练

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
y = LabelEncoder().fit_transform(all_image_types).reshape(-1)  # LabelEncoder：在0和n_class-1之间对标签进行编码
X = gray_imgs_mat # no need for normalizing, we already did this earlier Normalizer().fit_transform(gray_imgs_mat)
X.shape, y.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_train, y_test

In [ ]:
clf = LogisticRegression()
grid = {
    'C': [1e-9, 1e-6, 1e-3, 1e0],
    'penalty': ['l1', 'l2']
}
# 多CPU
cv = GridSearchCV(clf, grid, scoring='neg_log_loss', n_jobs=-1, verbose=1)
cv.fit(X_train, y_train)

In [ ]:
for i in range(1, len(cv.cv_results_['params'])+1):
    rank = cv.cv_results_['rank_test_score'][i-1]
    s = cv.cv_results_['mean_test_score'][i-1]
    sd = cv.cv_results_['std_test_score'][i-1]
    params = cv.cv_results_['params'][i-1]
    print("{0}. Mean validation neg log loss: {1:.6f} (std: {2:.6f}) - {3}".format(
        rank,
        s,
        sd,
        params
    ))
    
print('AFTER LR Parameters: ', cv.best_params_)
print("AFTER LR Training w/bin score mean: {:.2f}". format(cv.cv_results_['mean_train_score'][cv.best_index_]*100)) 
print("AFTER LR Test w/bin score mean: {:.2f}". format(cv.cv_results_['mean_test_score'][cv.best_index_]*100))
print("AFTER LR Test w/bin score 3*std: +/- {:.2f}". format(cv.cv_results_['std_test_score'][cv.best_index_]*100*3))
print('-'*10)

In [ ]:
y_test_hat_p = cv.predict_proba(X_test)  # 分类概率矩阵，每行和为1

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(y_test_hat_p[:,0], color='red')
sns.distplot(y_test_hat_p[:,1], color='blue')
sns.distplot(y_test_hat_p[:,2], color='green')

In [ ]:
dfy = pd.DataFrame({'0': y_test_hat_p[:,0], '1': y_test_hat_p[:,1], '2': y_test_hat_p[:,2]})
sns.pairplot(dfy) # 多变量图

## 4.2 混淆矩阵

混淆矩阵是二分类和多分类问题中一个标准的分析工具，在每个条目中，C_{i,j}对应的是具有正确类标签i并被预测为j的样本数目，。

我们可以看到，我们的模型有一个很大的问题，混淆了类0和类2。

In [ ]:
from sklearn.metrics import confusion_matrix

y_test_hat = cv.predict(X_test)

data = [
    go.Heatmap(
        z=confusion_matrix(y_test, y_test_hat),
        x=[0, 1, 2],
        y=[0, 1, 2],
        colorscale='Viridis',
        text = True ,
        opacity = 1.0
    )
]

layout = go.Layout(
    title='Test Confusion matrix',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 900, height = 700,
    
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

**混淆矩阵在训练集上**

In [ ]:
from sklearn.metrics import confusion_matrix

y_train_hat = cv.predict(X_train)

data = [
    go.Heatmap(
        z=confusion_matrix(y_train, y_train_hat),
        x=[0, 1, 2],
        y=[0, 1, 2],
        colorscale='Viridis',
        text = True ,
        opacity = 1.0
    )
]

layout = go.Layout(
    title='Training Confusion matrix',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 900, height = 700,
    
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

In [ ]:
查看训练集上的混淆矩阵，我们可以看到，我们几乎只预测了类1。